In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zOhjAr5Y7YpdOX6GvszR8glA6p1IuYVAtsg4Tcvl2MOs',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'carresalevaluepredictiondeploymen-donotdelete-pr-ryosh4pvhxemjh'
object_key = 'autos_preprocessed.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Unnamed: 0,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,1,18300,coupe,2011,manual,190,not-declared,125000,5,diesel,audi,Yes
1,2,9800,suv,2004,automatic,163,grand,125000,8,diesel,jeep,not-declared
2,3,1500,small car,2001,manual,75,golf,150000,6,petrol,volkswagen,No
3,4,3600,small car,2008,manual,69,fabia,90000,7,diesel,skoda,No
4,5,650,limousine,1995,manual,102,3er,150000,10,petrol,bmw,Yes


In [3]:


# print(df.seller.value_counts())
# df[df.seller!='gewerblich']
# # df=df.drop('seller',1)
# print(df.offerType.value_counts())
# df[df.offerType!='Gesuch']
# df=df.drop('offerType',1)

In [4]:
print(df.shape)
df=df[(df.powerPS >50) & (df.powerPS <900)]
df=df[(df.yearOfRegistration >=1950) & (df.yearOfRegistration <2017)]
print(df.shape)

(278578, 12)
(278578, 12)


In [5]:
# df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'],axis='columns',inplace=True)

In [6]:
new_df=df.copy()
new_df=new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])

In [7]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'),inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [8]:
new_df=new_df[(new_df.price >=100)&(new_df.price <=150000)]
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df['gearbox'].fillna(value='not-declared',inplace=True)
# new_df['vehicleType'].fillna(value='not-declared',inplace=True)
new_df['model'].fillna(value='not-declared',inplace=True)

In [9]:
# new_df.to_csv("autos_preprocessed.csv")

In [10]:
from sklearn.preprocessing import LabelEncoder
labels=['gearbox','notRepairedDamage','model','brand','fuelType','vehicleType']
mapper={}
for i in labels:
    mapper[i]=LabelEncoder()
    mapper[i].fit(new_df[i])
    tr=mapper[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'),mapper[i].classes_)
    print(i,":",mapper[i])
    new_df.loc[:,i+'_labels']=pd.Series(tr,index=new_df.index)
labeled = new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration']+[x+"_labels" for x in labels]]
print(labeled.columns)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [11]:
Y=labeled.iloc[:,0].values
X=labeled.iloc[:,1:].values
Y=Y.reshape(-1,1)

In [12]:
from sklearn.model_selection import cross_val_score,train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.3,random_state=3)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators=1000,max_depth=10,random_state=34)
regressor.fit(X_train,np.ravel(Y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

In [14]:
y_pred=regressor.predict(X_test)
print(r2_score(Y_test,y_pred))

0.834527626497731


In [15]:
filename='resale_model.sav'
pickle.dump(regressor,open(filename,'wb'))

In [16]:
!pip install ibm_watson_machine_learning

In [17]:
from ibm_watson_machine_learning import APIClient
wml_credentials = { 
                "url": "https://us-south.ml.cloud.ibm.com",
                "apikey": "IUutOrl_NmOCfCmq5LFkPOahP8Xbs435mg2ckAo5FubY"
}
client = APIClient(wml_credentials)

In [18]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space["resources"] if item["entity"]["name"] == space_name)["metadata"]["id"])

In [19]:
space_uid = guid_from_space_name(client, "models")
print("Space UID = " + space_uid)

Space UID = bfb96438-93c4-4905-8671-1f4532e811c5


In [20]:
client.set.default_space(space_uid)

'SUCCESS'

In [21]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [22]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [23]:
model_details = client.repository.store_model(model=regressor,meta_props={
    client.repository.ModelMetaNames.NAME : "Model Building",
    client.repository.ModelMetaNames.TYPE : "scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                             )

model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [24]:
model_id

'11f603fb-8bf7-42d5-8e67-56ea74c738de'